<a href="https://colab.research.google.com/github/asadali163/NLP-Transformers/blob/main/codebase/NLP_Name_Entity_Recognitio_basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget -nc https://lazyprogrammer.me/course_files/nlp/ner_train.pkl
!wget -nc https://lazyprogrammer.me/course_files/nlp/ner_test.pkl

--2024-08-03 14:55:05--  https://lazyprogrammer.me/course_files/nlp/ner_train.pkl
Resolving lazyprogrammer.me (lazyprogrammer.me)... 172.67.213.166, 104.21.23.210, 2606:4700:3031::6815:17d2, ...
Connecting to lazyprogrammer.me (lazyprogrammer.me)|172.67.213.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4757208 (4.5M) [application/octet-stream]
Saving to: ‘ner_train.pkl’

ner_train.pkl       100%[===================>]   4.54M  --.-KB/s    in 0.1s    

2024-08-03 14:55:06 (45.7 MB/s) - ‘ner_train.pkl’ saved [4757208/4757208]

--2024-08-03 14:55:06--  https://lazyprogrammer.me/course_files/nlp/ner_test.pkl
Resolving lazyprogrammer.me (lazyprogrammer.me)... 172.67.213.166, 104.21.23.210, 2606:4700:3031::6815:17d2, ...
Connecting to lazyprogrammer.me (lazyprogrammer.me)|172.67.213.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1201978 (1.1M) [application/octet-stream]
Saving to: ‘ner_test.pkl’

ner_test.pkl        100%[=========

In [ ]:
!pip install transformers

In [ ]:
from transformers import pipeline
import pickle
import numpy as np

In [ ]:
ner = pipeline("ner", aggregation_strategy="simple")

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/998 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
with open("ner_train.pkl", "rb") as f:
    train_data = pickle.load(f)
with open("ner_test.pkl", "rb") as f:
    test_data = pickle.load(f)

In [ ]:
test_data[0]

[('CRICKET', 'O'),
 ('-', 'O'),
 ('LEICESTERSHIRE', 'B-ORG'),
 ('TAKE', 'O'),
 ('OVER', 'O'),
 ('AT', 'O'),
 ('TOP', 'O'),
 ('AFTER', 'O'),
 ('INNINGS', 'O'),
 ('VICTORY', 'O'),
 ('.', 'O')]

In [ ]:
len(test_data), len(test_data[30])

(2970, 13)

As it contain 2970 sentences, each of some length

In [ ]:
# Let's seperate the tag and targets
inputs, targets = [], []

for item in test_data:
  token, target = [], []
  for word in item:
    token.append(word[0])
    target.append(word[1])
  inputs.append(token)
  targets.append(target)

In [ ]:
# Let's see at one input
inputs[20]

['Chester-le-Street',
 ':',
 'Glamorgan',
 '259',
 'and',
 '207',
 '(',
 'A.',
 'Dale',
 '69',
 ',',
 'H.',
 'Morris',
 '69',
 ';',
 'D.',
 'Blenkiron',
 '4-43',
 ')',
 ',',
 'Durham',
 '114',
 '(',
 'S.',
 'Watkin',
 '4-28',
 ')',
 'and',
 '81-3',
 '.']

In [ ]:
len(inputs[34]), len(targets[34])

(8, 8)

As these models work with complete sentences, not with single word. So we need to join the full sentences using nltk

In [ ]:
from nltk.tokenize.treebank import TreebankWordDetokenizer

In [ ]:
detokenizer = TreebankWordDetokenizer()

In [ ]:
detokenizer.detokenize(inputs[20])

'Chester-le-Street: Glamorgan 259 and 207 (A. Dale 69, H. Morris 69; D. Blenkiron 4-43), Durham 114 (S. Watkin 4-28) and 81-3.'

In [ ]:
targets[20]

['B-LOC',
 'O',
 'B-ORG',
 'O',
 'O',
 'O',
 'O',
 'B-PER',
 'I-PER',
 'O',
 'O',
 'B-PER',
 'I-PER',
 'O',
 'O',
 'B-PER',
 'I-PER',
 'O',
 'O',
 'O',
 'B-ORG',
 'O',
 'O',
 'B-PER',
 'I-PER',
 'O',
 'O',
 'O',
 'O',
 'O']

In [ ]:
ner(detokenizer.detokenize(inputs[20]))

[{'entity_group': 'LOC',
  'score': 0.9848647,
  'word': 'Chester - le - Street',
  'start': 0,
  'end': 17},
 {'entity_group': 'ORG',
  'score': 0.99960023,
  'word': 'Glamorgan',
  'start': 19,
  'end': 28},
 {'entity_group': 'PER',
  'score': 0.9965984,
  'word': 'A. Dale',
  'start': 42,
  'end': 49},
 {'entity_group': 'PER',
  'score': 0.997134,
  'word': 'H. Morris',
  'start': 54,
  'end': 63},
 {'entity_group': 'PER',
  'score': 0.9164668,
  'word': 'D. Blenkiron',
  'start': 68,
  'end': 80},
 {'entity_group': 'ORG',
  'score': 0.9995956,
  'word': 'Durham',
  'start': 88,
  'end': 94},
 {'entity_group': 'PER',
  'score': 0.9938167,
  'word': 'S. Watkin',
  'start': 100,
  'end': 109}]